# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
# imports df from csv file from part 1
cities_df = pd.read_csv("../WeatherPy/cities.csv")
cities_df

,Unnamed: 0,City,Cloudiness,Humidity,Country,Date,Latitude,Longitude,Max Temperature,Wind Speed
0,0,Yerbogachën,83,87,RU,1585959573,61.28,108.01,30.76,14.16
1,1,Nikolskoye,90,74,RU,1585959767,59.70,30.79,33.80,11.18
2,2,Sola,100,87,VU,1585959767,-13.88,167.55,79.09,22.68
3,3,Cidreira,22,74,BR,1585959767,-30.18,-50.21,67.44,15.88
4,4,Saint-Pierre,90,94,RE,1585959767,-21.34,55.48,73.40,9.17
...,...,...,...,...,...,...,...,...,...,...
548,548,Tisul',53,95,RU,1585960626,55.76,88.31,29.66,2.28
549,549,Banda Aceh,97,83,ID,1585960627,5.56,95.32,80.15,7.78
550,550,Barhi,32,17,IN,1585960628,23.90,80.80,74.48,2.71
551,551,Burnie,100,70,AU,1585960629,-41.07,145.92,61.00,14.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# creates map
fig = gmaps.figure()
# format heatmap layer
heatmap_layer = gmaps.heatmap_layer(
    cities_df[["Latitude", "Longitude"]], weights = cities_df["Humidity"], max_intensity=cities_df["Humidity"].max(), point_radius=5.0)
# adds heatmap layer to original map
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# creates new df with only ideal conditions (temp between 75 and 85, wind speed less than 10, no clouds)
ideal_df = cities_df.loc[(cities_df["Max Temperature"] > 75) & (cities_df["Max Temperature"] <= 85) & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0)]
ideal_df


,Unnamed: 0,City,Cloudiness,Humidity,Country,Date,Latitude,Longitude,Max Temperature,Wind Speed
95,95,Canavieiras,0,83,BR,1585959783,-15.68,-38.95,79.63,8.81
106,106,Muscat,0,73,OM,1585959643,23.61,58.59,77.00,1.12
213,213,Adrar,0,15,MR,1585959800,20.50,-10.07,76.01,6.26
315,315,Islamkot,0,28,PK,1585959812,24.70,70.18,78.21,4.76
378,378,Abu Dhabi,0,50,AE,1585959631,24.47,54.37,79.00,1.12
483,483,Muhafazat Lahij,0,69,YE,1585960552,13.17,44.58,75.22,7.31
524,524,Kolokani,0,14,ML,1585960600,13.57,-8.03,84.94,9.26
547,547,Daman,0,65,IN,1585960625,20.42,72.85,77.56,2.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# creates new empty data frame

ideal_df["Hotel Name"] = ""
ideal_df["Hotel Lat"] = ""
ideal_df["Hotel Lng"] = ""
ideal_df


/opt/anaconda3/envs/PythonData2020_1/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/envs/PythonData2020_1/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/envs/PythonData2020_1/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value

,Unnamed: 0,City,Cloudiness,Humidity,Country,Date,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
95,95,Canavieiras,0,83,BR,1585959783,-15.68,-38.95,79.63,8.81,,,
106,106,Muscat,0,73,OM,1585959643,23.61,58.59,77.00,1.12,,,
213,213,Adrar,0,15,MR,1585959800,20.50,-10.07,76.01,6.26,,,
315,315,Islamkot,0,28,PK,1585959812,24.70,70.18,78.21,4.76,,,
378,378,Abu Dhabi,0,50,AE,1585959631,24.47,54.37,79.00,1.12,,,
483,483,Muhafazat Lahij,0,69,YE,1585960552,13.17,44.58,75.22,7.31,,,
524,524,Kolokani,0,14,ML,1585960600,13.57,-8.03,84.94,9.26,,,
547,547,Daman,0,65,IN,1585960625,20.42,72.85,77.56,2.59,,,


In [8]:
# sets constant parameters for url
params = {
    "keyword": "hotel",
    "radius": 5,
    "type": "hotel",
    "key": g_key
}

# iterates through every index and and row
for index, row in ideal_df.iterrows():
# adds timer on loop so I don't go over my API limit
    time.sleep(1)
# sets the coordinates of each city to a variable
    location = "{0},{1}".format(row["Latitude"], row["Longitude"])
# adds coordinates to params
    params["location"] = location
# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# adds params to url
    response = requests.get(base_url, params=params).json()
# if a hotel exists within 5000 m, store its name, lat, and lng into df. If not, skip it
    try:
        ideal_df.loc[index, 'Hotel Name'] = response["results"][0]["name"]
        ideal_df.loc[index, 'Hotel Lat'] = response["results"][0]["geometry"]["location"]["lat"]
        ideal_df.loc[index, 'Hotel Lng'] = response["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        print("No hotels in this radius... skipping.")


No hotels in this radius... skipping.
No hotels in this radius... skipping.
No hotels in this radius... skipping.


/opt/anaconda3/envs/PythonData2020_1/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


No hotels in this radius... skipping.
No hotels in this radius... skipping.
No hotels in this radius... skipping.


In [16]:
ideal_df

,Unnamed: 0,City,Cloudiness,Humidity,Country,Date,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
95,95,Canavieiras,0,83,BR,1585959783,-15.68,-38.95,79.63,8.81,,,
106,106,Muscat,0,73,OM,1585959643,23.61,58.59,77.00,1.12,,,
213,213,Adrar,0,15,MR,1585959800,20.50,-10.07,76.01,6.26,,,
315,315,Islamkot,0,28,PK,1585959812,24.70,70.18,78.21,4.76,Ghazi Guest House,24.7006,70.178
378,378,Abu Dhabi,0,50,AE,1585959631,24.47,54.37,79.00,1.12,"Marriott Hotel Downtown, Abu Dhabi",24.4704,54.3697
483,483,Muhafazat Lahij,0,69,YE,1585960552,13.17,44.58,75.22,7.31,,,
524,524,Kolokani,0,14,ML,1585960600,13.57,-8.03,84.94,9.26,,,
547,547,Daman,0,65,IN,1585960625,20.42,72.85,77.56,2.59,,,


In [15]:
# makes new df with only the cities that have hotels within 5000 m
hotel_df = ideal_df.loc[ideal_df["Hotel Name"] != ""]
hotel_df


,Unnamed: 0,City,Cloudiness,Humidity,Country,Date,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
315,315,Islamkot,0,28,PK,1585959812,24.70,70.18,78.21,4.76,Ghazi Guest House,24.7006,70.178
378,378,Abu Dhabi,0,50,AE,1585959631,24.47,54.37,79.00,1.12,"Marriott Hotel Downtown, Abu Dhabi",24.4704,54.3697


In [18]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [21]:
# set layout properties
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…